# Model 1 - MCMC with Turing.jl

## Install Turing.jl (only need to do this once)
See the Turing instructions on their [wiki](https://github.com/yebai/Turing.jl/wiki/Get-started#install-turingjl) for more info.

In [1]:
#Pkg.add("Turing")
#Pkg.update()
#Pkg.checkout("Turing", "master") 
#Pkg.build("Turing")
#Pkg.test("Turing")

## Setup

In [2]:
using Turing

[Turing]: AD chunk size is set as 40


In [3]:
using Distributions, StatsFuns
using Plots
using Mamba: describe, plot

## STEP 1: define data

In [4]:
data = Dict()
data["T"] = 100
data["k"] = [50, 51, 57, 55, 63, 62, 82, 94, 99, 100]
data["Δμ"] = [0.0100, 0.0215, 0.0464, 0.1000, 0.2154, 0.4642, 1.0000, 2.1544, 4.6416, 10.0000];

In [5]:
plot_data_space = scatter(log.(data["Δμ"]), data["k"]./data["T"],
    xlabel = "log signal instensity, Δμ", #xscale = :log,
    ylabel = "proportion correct, k/T",
    legend = false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -4 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 0.5 
 
 
 0.6 
 
 
 0.7 
 
 
 0.8 
 
 
 0.9 
 
 
 1.0 
 
 
 log signal instensity, ?? 
 
 
 proportion correct, k/T

## STEP 2: conduct inference
First we are going to define a model. This is the way of doing it for the Turing package. I managed to convert this over from the JAGS specification (see the original Matlab repo) with not so much trouble. It's pretty nice in fact.

In [6]:
@model model1(k, Δμ, T) = begin
    Φ(x) = normcdf(0, 1, x)
    PC(Δμ, σ²) = Φ(Δμ/sqrt(2σ²))
    σ² ~ Uniform(0,100)
    for c ∈ 1:length(k)
        k[c] ~ Binomial(T, PC(Δμ[c],σ²))
    end
    return σ²
end

model1 (generic function with 4 methods)

Now do sampling. Note that there are a variety of sampling algorithms we can use.

In [7]:
n_samples = 300
samples = sample( model1(data["k"], data["Δμ"], data["T"]), PG(50,n_samples))
σ²posterior = samples[:σ²]

# TODO: plot histogram of σ²posterior

[Turing]:  Assume - `σ²` is a parameter
 @~(::ANY, ::ANY) at compiler.jl:76
[Turing]:  Observe - `k` is an observation
 @~(::ANY, ::ANY) at compiler.jl:57


[PG] Sampling...  0%  ETA: 0:05:11

[PG] Finished with
  Running time    = 1.9845896890000014;


[PG] Sampling...100% Time: 0:00:02


300-element Array{Float64,1}:
 1.82396 
 1.44559 
 1.44559 
 1.44559 
 1.44559 
 1.44559 
 1.44559 
 1.44559 
 1.44559 
 0.934792
 0.934792
 0.934792
 0.934792
 ⋮       
 1.25379 
 1.25379 
 1.25379 
 1.25379 
 1.25379 
 1.25379 
 1.25379 
 1.25379 
 1.25379 
 1.25379 
 0.986988
 0.986988

## STEP 3: posterior prediction
We'll now write our own function to do posterior prediction given the posterior samples of `σ²`

In [8]:
function posterior_prediction(T::Int64, Δμ_list, σ²samples)
    Φ(x) = normcdf(0, 1, x)
    PC(Δμ, σ²) = Φ(Δμ/sqrt(2σ²))
    k_pred = zeros(n_samples, length(data["Δμ"]))
    k_pred = [rand(Binomial(T, PC(Δμ,σ²))) for Δμ ∈ Δμ_list, σ² ∈ σ²samples]
end

k_pred = posterior_prediction(data["T"], data["Δμ"], σ²posterior)
# so now we have posterior predictive samples in k_pred.
# Each row corresponds to an mcmc sample, each column corresponds to a value of Δμ


# TODO: do summary stats or plotting of posterior predictions, and plot the actual data so we
# can do model critique visually

# TODO: can also do out of sample prediction simply by providing more values for Δμ

10×300 Array{Int64,2}:
  45   54   45   58   53   45   50   52  …   47   48   65   51   48   45   45
  48   43   45   49   63   50   45   47      62   50   46   58   54   50   52
  54   49   48   49   49   49   46   56      48   56   61   47   56   59   60
  46   49   56   51   49   49   49   47      50   54   52   54   53   49   50
  56   57   58   59   54   51   58   57      58   45   53   52   58   68   60
  61   57   59   51   57   60   55   59  …   67   57   62   62   67   60   61
  70   81   84   77   76   74   68   70      77   76   69   72   64   82   81
  79   92   91   89   89   91   88   86      93   91   89   93   96   95   96
  98   98   99  100  100  100  100  100     100   99  100  100  100  100  100
 100  100  100  100  100  100  100  100     100  100  100  100  100  100  100

## References
Vincent, B. T. (2015). A tutorial on Bayesian models of perception. Journal of Mathematical Psychology, 66, 103–114. http://doi.org/10.1016/j.jmp.2015.02.001